## 4장. 결합

In [1]:
import pandas as pd
import numpy as np

In [2]:
# warning 무시
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
reserve_tb=pd.read_csv('./data/reserve.csv',encoding='UTF-8')
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


In [19]:
hotel_tb = pd.read_csv('./data/hotel.csv', encoding='UTF-8')
hotel_tb.head()

,hotel_id,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,h_1,26100,D,D-2,43.064569,141.511397,True
1,h_2,26400,A,A-1,35.715320,139.939446,True
2,h_3,41300,E,E-4,35.281572,136.988565,False
3,h_4,5200,C,C-3,38.431293,140.795615,False
4,h_5,13500,G,G-3,33.597291,130.533872,True


In [21]:
customer_tb = pd.read_csv('./data/customer.csv', encoding='UTF-8')
customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude
0,c_1,41,man,35.092193,136.512347
1,c_2,38,man,35.325076,139.410551
2,c_3,49,woman,35.120543,136.511179
3,c_4,43,man,43.034868,141.240314
4,c_5,31,man,35.102661,136.523797


-------
## 4.1 마스터 테이블에서 정보얻기
예약 테이블과 호텔 테이블을 결합해 숙박인원수가 한 명인 비즈니스호텔의 예약 레코드를 추출해보자.

#### not Awesome

In [10]:
pd.merge(reserve_tb, hotel_tb, on='hotel_id', how='inner').query('people_num==1 & is_business').head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
14,r310,h_219,c_73,2017-03-30 01:52:36,2017-04-18,11:00:00,2017-04-21,1,30900,10300,B,B-3,35.644729,139.693389,True
18,r1433,h_219,c_348,2017-04-08 03:19:47,2017-04-25,10:00:00,2017-04-28,1,30900,10300,B,B-3,35.644729,139.693389,True
21,r2992,h_219,c_756,2016-02-17 14:02:23,2016-03-01,12:00:00,2016-03-03,1,20600,10300,B,B-3,35.644729,139.693389,True
24,r3522,h_219,c_875,2016-09-09 07:10:21,2016-09-20,10:00:00,2016-09-21,1,10300,10300,B,B-3,35.644729,139.693389,True
101,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,34500,C,C-1,38.237294,140.696131,True


#### Awesome
Not Awesome 에서와 달리, 결합처리 전에 조건에맞는 데이터를 추출한 다음 결합함

In [14]:
pd.merge(reserve_tb.query('people_num==1'), hotel_tb.query('is_business'), on='hotel_id', how='inner').head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,34500,C,C-1,38.237294,140.696131,True
1,r997,h_256,c_244,2016-10-15 22:47:40,2016-10-31,10:30:00,2016-11-02,1,69000,34500,C,C-1,38.237294,140.696131,True
2,r2602,h_256,c_650,2016-05-10 00:42:56,2016-05-12,11:00:00,2016-05-14,1,69000,34500,C,C-1,38.237294,140.696131,True
3,r3738,h_256,c_930,2017-04-12 09:53:00,2017-05-08,11:30:00,2017-05-09,1,34500,34500,C,C-1,38.237294,140.696131,True
4,r11,h_183,c_2,2016-11-19 12:49:10,2016-12-08,11:00:00,2016-12-11,1,29700,9900,G,G-4,33.595248,130.633567,True


--------------
## 4.2 조건에 따라 결합할 마스터 테이블 변경하기
소규모 지역 단위 (small_area_name)에 추천 후보 수가 20건 이상이면, 같은 소규모 지역 단위의 호텔을 후보로 하고, <br>
같은 소규모 지역 단위의 호텔 건수가 20건 미만이면 같은 대규모 지역단위 (big_area_name)의 호텔을 추천 후보로 한다.

> #### python skill
concat() 과 merge() 차이점 구분
* merge는 2개의 데이터를 콤마를 이용하여 넣어줬지만, concat은 리스트형식으로 연결.

In [14]:
# 가비지 컬렉션 (불필요한 메모리 해제)을 ㅜ이한 라이브러리
import gc

# small_area_name별로 호텔 수 계산
small_area_mst = hotel_tb.groupby(['big_area_name','small_area_name'], as_index=False).size().reset_index()
small_area_mst.columns=['big_area_name','small_area_name','hotel_cnt']

In [15]:
small_area_mst.head()

,big_area_name,small_area_name,hotel_cnt
0,A,A-1,35
1,A,A-3,30
2,B,B-1,15
3,B,B-2,18
4,B,B-3,19


In [16]:
# 20건 이상이면 join_area_id를 small_area_name으로서 지정.
# 20건 미만이면 join_area_id를 big_area_name으로서 지정.
# -1로 자신을 제외

small_area_mst['join_area_id']=np.where(small_area_mst['hotel_cnt']-1 >=20, 
                                        small_area_mst['small_area_name'],
                                        small_area_mst['big_area_name'])

In [17]:
# 필요 없어진 열 삭제
small_area_mst.drop(['hotel_cnt','big_area_name'],axis=1, inplace=True)

In [18]:
small_area_mst.head()

,small_area_name,join_area_id
0,A-1,A-1
1,A-3,A-3
2,B-1,B
3,B-2,B
4,B-3,B


In [22]:
# 추천 전 호텔에 small_area_mst를 결합하여 join_area_id를 설정.
base_hotel_mst=pd.merge(hotel_tb, small_area_mst, on='small_area_name').loc[:,['hotel_id','join_area_id']]
base_hotel_mst.head()

,hotel_id,join_area_id
0,h_1,D
1,h_79,D
2,h_125,D
3,h_127,D
4,h_129,D


In [ ]:
# # 메모리에 여유가 없을 때 메모리 해제
# del small_area_mst
# gc.collect()

In [20]:
# recommend_hotel_mst는 추천 후보 테이블
## concat이므로, 위 아래로 두 테이블을 붙인것.
recommend_hotel_mst=pd.concat([hotel_tb[['small_area_name','hotel_id']].rename(columns={'small_area_name':'join_area_id'}, inplace=False),
          hotel_tb[['big_area_name','hotel_id']].rename(columns={'big_area_name':'join_area_id'}, inplace=False)])
recommend_hotel_mst.head()

,join_area_id,hotel_id
0,D-2,h_1
1,A-1,h_2
2,E-4,h_3
3,C-3,h_4
4,G-3,h_5


In [24]:
# 결합할 대 hotel_id 열 이름이 중복되므로 변경.
recommend_hotel_mst.rename(columns={'hotel_id':'rec_hotel_id'}, inplace=True)

In [25]:
recommend_hotel_mst.head()

,join_area_id,rec_hotel_id
0,D-2,h_1
1,A-1,h_2
2,E-4,h_3
3,C-3,h_4
4,G-3,h_5


In [43]:
# base_hotel_mst와 recommend_hotel_mst를 결합하여 추천 후보의 정보를 부여한다.
## query함수로 추천 후보에서 자신을 제외.
pd.merge(base_hotel_mst, recommend_hotel_mst, on='join_area_id').loc[:,['hotel_id','rec_hotel_id']].query('hotel_id != rec_hotel_id')

,hotel_id,rec_hotel_id
1,h_1,h_14
2,h_1,h_22
3,h_1,h_27
4,h_1,h_40
5,h_1,h_45
...,...,...
9659,h_220,h_226
9660,h_220,h_249
9661,h_220,h_271
9662,h_220,h_281


---------------
## 4.3 과거 데이터에서 정보 얻기
고객이 다음에 예약할 호텔의 가격대를 예측하고 싶어하는 상황 <br>
먼저 고객이 과거에 예약한 호텔의 가격대를 통해 예약할 호텔의 가격대를 예측할 수 있다는 가설을 세운다. 

> #### python skill
* 그룹별로 shift된 결과 받아오기
    * data['column'].groupby().shift(periods=n)

In [20]:
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


### Q1. n건 전의 데이터 얻기

In [26]:
# groupby한 데이터를 특정 컬럼 기준으로 sort_values적용
result=reserve_tb.groupby('customer_id').apply(lambda group: group.sort_values(by='reserve_datetime', axis=0, inplace=False))
result.head(10)

reserve_id hotel_id customer_id     reserve_datetime  \
customer_id                                                           
c_1         0          r1     h_75         c_1  2016-03-06 13:09:42   
            1          r2    h_219         c_1  2016-07-16 23:39:55   
            2          r3    h_179         c_1  2016-09-24 10:03:17   
            3          r4    h_214         c_1  2017-03-08 03:20:10   
            4          r5     h_16         c_1  2017-09-05 19:50:37   
            5          r6    h_241         c_1  2017-11-27 18:47:05   
            6          r7    h_256         c_1  2017-12-29 10:38:36   
            7          r8    h_241         c_1  2018-05-26 08:42:51   
c_10        58        r59    h_245        c_10  2016-06-20 11:10:52   
            59        r60    h_102        c_10  2016-10-23 01:56:42   

               checkin_date checkin_time checkout_date  people_num  \
customer_id                                                          
c_1         0    2016-03-26     10:00:00    2016-03-29           4   
            1    2016-07-20     11:30:00    2016-07-21           2   
            2    2016-10-19     09:00:00    2016-10-22           2   
            3    2017-03-29     11:00:00    2017-03-30           4   
            4    2017-09-22     10:30:00    2017-09-23           3   
            5    2017-12-04     12:00:00    2017-12-06           3   
            6    2018-01-25     10:30:00    2018-01-28           1   
            7    2018-06-08     10:00:00    2018-06-09           1   
c_10        58   2016-07-08     11:00:00    2016-07-10           4   
            59   2016-11-01     11:00:00    2016-11-03           3   

                total_price  
customer_id                  
c_1         0         97200  
            1         20600  
            2         33600  
            3        194400  
            4         68100  
            5         36000  
            6        103500  
            7          6000  
c_10        58       110400  
            59        36000

In [28]:
result['before_price']=result['total_price'].groupby('customer_id').shift(periods=2)
result.head(10)

reserve_id hotel_id customer_id     reserve_datetime  \
customer_id                                                           
c_1         0          r1     h_75         c_1  2016-03-06 13:09:42   
            1          r2    h_219         c_1  2016-07-16 23:39:55   
            2          r3    h_179         c_1  2016-09-24 10:03:17   
            3          r4    h_214         c_1  2017-03-08 03:20:10   
            4          r5     h_16         c_1  2017-09-05 19:50:37   
            5          r6    h_241         c_1  2017-11-27 18:47:05   
            6          r7    h_256         c_1  2017-12-29 10:38:36   
            7          r8    h_241         c_1  2018-05-26 08:42:51   
c_10        58        r59    h_245        c_10  2016-06-20 11:10:52   
            59        r60    h_102        c_10  2016-10-23 01:56:42   

               checkin_date checkin_time checkout_date  people_num  \
customer_id                                                          
c_1         0    2016-03-26     10:00:00    2016-03-29           4   
            1    2016-07-20     11:30:00    2016-07-21           2   
            2    2016-10-19     09:00:00    2016-10-22           2   
            3    2017-03-29     11:00:00    2017-03-30           4   
            4    2017-09-22     10:30:00    2017-09-23           3   
            5    2017-12-04     12:00:00    2017-12-06           3   
            6    2018-01-25     10:30:00    2018-01-28           1   
            7    2018-06-08     10:00:00    2018-06-09           1   
c_10        58   2016-07-08     11:00:00    2016-07-10           4   
            59   2016-11-01     11:00:00    2016-11-03           3   

                total_price  before_price  
customer_id                                
c_1         0         97200           NaN  
            1         20600           NaN  
            2         33600       97200.0  
            3        194400       20600.0  
            4         68100       33600.0  
            5         36000      194400.0  
            6        103500       68100.0  
            7          6000       36000.0  
c_10        58       110400           NaN  
            59        36000           NaN

#### apply, lambda 연습
* 기존 컬럼의 집계값을 새로운 컬럼으로 추가하고 싶을 때 활용가능

In [29]:
df3 = pd.DataFrame({
    'A':[1,3,4,3,4],
    'B':[2,3,1,2,3],
    'C':[1,5,2,4,4]
})
df3

,A,B,C
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


In [30]:
# 각 열의 최대값과 최소값의 차이
df3.apply(lambda x: x.max()-x.min())

A    3
B    2
C    4
dtype: int64

In [31]:
# 각 행의 최대값과 최소값의 차이
df3.apply(lambda x:x.max()-x.min(), axis=1)

0    1
1    2
2    3
3    2
4    1
dtype: int64

In [4]:
aa=pd.DataFrame({'id':['a','a','a','b','b'],
                'val':[1,2,3,4,5]})
aa

,id,val
0,a,1
1,a,2
2,a,3
3,b,4
4,b,5


In [9]:
# groupby() 를 하여 apply(lambda) 함수로 DataFrame내 각 컬럼별 요약 집계 결과를 새로운 DataFrame을 만들어 보는 예제
aa.groupby('id').apply(lambda x: pd.Series({'val_sum':sum(x.val), 'val_cnt':len(x), 'id_list':x.id.tolist()}))

,val_sum,val_cnt,id_list
id,,,
a,6,3,"[a, a, a]"
b,9,2,"[b, b]"


### Q2.  과거 n건의 합곗값
두 건 이전부터 자신을 포함한 세 건의 합계 계산.

> #### python skill
* rolling( ) 함수
    * rolling함수는 데이터를 window로 나누는 함수
    * window 매개변수로 자신을 포함하여 몇 건을 대상으로 할지 설정
    * min_periods 매개변수를 설정하여 설정한 값 이상의 건수를 만족했을 때만 계산을 실행하며, 설정한 건수에 미치지 못할 경우에는 NAN이 된다.
    * center를 True로 하면, 자신의 데이터 행이 가운데 오도록 window를 선택하고, <br>
    center를 설정하지 않으면 자신의 데이터 행을 포함하여 설정한 건수가 되도록 아래의 데이터 행을 더한다. 

In [44]:
# customer_id별로 reserve_datetime으로 데이터 정렬
result=reserve_tb.groupby('customer_id').apply(lambda x: x.sort_values(by='reserve_datetime', ascending=True)).reset_index(drop=True)
result.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


In [73]:
# 새로운 열로 price_sum을 추가
result['price_sum']=pd.Series(result.loc[:,["customer_id","total_price"]]
                              
        # customer_id별로 total_price의 window 3건의 합곗값을 계산
          .groupby('customer_id')
          .rolling(center=False, window=3, min_periods=3).sum()
                              
        # group화를 해제하면서 total_price의 열을 구한다.
         .reset_index(drop=True)
         .loc[:,'total_price'])
result.head(10)

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_sum
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,NaN
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,NaN
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,151400.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,248600.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,296100.0
5,r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,298500.0
6,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,207600.0
7,r8,h_241,c_1,2018-05-26 08:42:51,2018-06-08,10:00:00,2018-06-09,1,6000,145500.0
8,r59,h_245,c_10,2016-06-20 11:10:52,2016-07-08,11:00:00,2016-07-10,4,110400,NaN
9,r60,h_102,c_10,2016-10-23 01:56:42,2016-11-01,11:00:00,2016-11-03,3,36000,NaN


### Q3. 과거 n건의 평균값
자신의 행을 포함하지 않고, 한 건 이전의 데이터에서 세 건 이전까지의 평균 예약 금액을 첨부하고, <br>
과거 예약 건수가 세 건 미만이면 존재하는 건수의 평균값을 계산한다.

> #### python skill
* rolling( ) 함수는 자신의 데이터 행을 포함하여 연산함. 
    * 따라서 이경우 shift( ) 함수로 데이터 행 이동시킴.

In [25]:
result = reserve_tb.groupby('customer_id').apply(lambda x: x.sort_values(by='reserve_datetime', ascending=True)).reset_index(drop=True)
result.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


In [26]:
result['price_avg']=pd.Series(result.groupby('customer_id')['total_price'].rolling(center=False, window=3, min_periods=1).mean().reset_index(drop=True))
result.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_avg
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,97200.000000
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,58900.000000
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,50466.666667
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,82866.666667
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,98700.000000


In [24]:
result['price_avg']=result.groupby('customer_id')['price_avg'].shift(periods=1)
result.head(10)

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_avg
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,NaN
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,97200.000000
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,58900.000000
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,50466.666667
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,82866.666667
5,r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,98700.000000
6,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,99500.000000
7,r8,h_241,c_1,2018-05-26 08:42:51,2018-06-08,10:00:00,2018-06-09,1,6000,69200.000000
8,r59,h_245,c_10,2016-06-20 11:10:52,2016-07-08,11:00:00,2016-07-10,4,110400,NaN
9,r60,h_102,c_10,2016-10-23 01:56:42,2016-11-01,11:00:00,2016-11-03,3,36000,110400.000000


### Q4. 과거 n일의 합곗값
예약 테이블의 모든 데이터에 자신을 포함하지 않으면서 같은 고객의 지난 90일간의 합계 예약 금액 정보를 첨부한다.<br>
예약이 하나도 없으면 값을 0으로 한다.

> #### python skill
* pd.to_datetime() 으로 날짜를 문자형에서 일시형으로 변환
* operator의 and_함수 사용 (&조건을 의미)
* offsets.Day() 
    * pandas.tseries.offsets 으로부터 일(Day), 시간(Hour), 분(Minute), 초(Minute) date offsets을 불러와서,<br>
    freq = Day(3)과 같이 Day(3)의 date offset으로 위의 freq = '3d'와 동일한 결과를 얻었습니다.  <br>
    => 3일 간격으로 날짜 생성할 때 사용
    * 이 경우에는 90일 전 날짜를 생성하기 위해 아래와 같이 작성
        * offsets.Day(-90)  <br>
        
    출처: https://rfriend.tistory.com/503

In [11]:
import pandas.tseries.offsets as offsets
import operator

In [12]:
# 시간 계산에 사용하기 위해 자료형을 문자형에서 일시형으로 변환.
reserve_tb['reserve_datetime']=pd.to_datetime(reserve_tb['reserve_datetime'],format='%Y-%m-%d %H:%M:%S')

In [13]:
# reserve_datetime의 날짜를 확인하지 않고, customer_id가 같은 데이터들을 모두 결합.
sum_table=pd.merge(reserve_tb[['reserve_id','customer_id','reserve_datetime']],
                  reserve_tb[['customer_id','reserve_datetime','total_price']].rename(columns={'reserve_datetime':'reserve_datetime_before'}),on='customer_id')
sum_table.head()

,reserve_id,customer_id,reserve_datetime,reserve_datetime_before,total_price
0,r1,c_1,2016-03-06 13:09:42,2016-03-06 13:09:42,97200
1,r1,c_1,2016-03-06 13:09:42,2016-07-16 23:39:55,20600
2,r1,c_1,2016-03-06 13:09:42,2016-09-24 10:03:17,33600
3,r1,c_1,2016-03-06 13:09:42,2017-03-08 03:20:10,194400
4,r1,c_1,2016-03-06 13:09:42,2017-09-05 19:50:37,68100


In [14]:
# reserve_datetime_before가 reserve_datetime보다 이전이면서 & 90일간 이니까, 90일전일자보단 커야됨.
sum_table[operator.and_(sum_table['reserve_datetime']>sum_table['reserve_datetime_before'],
                                       sum_table['reserve_datetime']+offsets.Day(-90)<= sum_table['reserve_datetime_before'])].head(10)

,reserve_id,customer_id,reserve_datetime,reserve_datetime_before,total_price
17,r3,c_1,2016-09-24 10:03:17,2016-07-16 23:39:55,20600
44,r6,c_1,2017-11-27 18:47:05,2017-09-05 19:50:37,68100
53,r7,c_1,2017-12-29 10:38:36,2017-11-27 18:47:05,36000
117,r15,c_2,2018-04-19 11:25:00,2018-02-18 05:12:58,75600
126,r16,c_2,2018-07-06 04:18:28,2018-04-19 11:25:00,68800
145,r19,c_3,2017-01-11 22:54:09,2016-10-22 02:18:48,20400
154,r20,c_3,2017-02-23 07:10:30,2017-01-11 22:54:09,390600
162,r21,c_3,2017-04-06 18:12:10,2017-01-11 22:54:09,390600
163,r21,c_3,2017-04-06 18:12:10,2017-02-23 07:10:30,18200
200,r26,c_4,2016-05-11 17:48:07,2016-03-28 07:17:34,39300


In [15]:
# checkin의 날짜를 비교하여 90일 이내 데이터가 결합된 데이터만을 추출.
## operator 의 and_ 함수로 결합조건 지정. (operator.and_ 는 & 연산을 의미)
# reserve_id별로 total_price의 합을 계산
sum_table=sum_table[operator.and_(sum_table['reserve_datetime']>sum_table['reserve_datetime_before'],
                                       sum_table['reserve_datetime']+offsets.Day(-90)<= sum_table['reserve_datetime_before'])].groupby('reserve_id')['total_price'].sum().reset_index()
sum_table.head()

,reserve_id,total_price
0,r1000,44700
1,r1004,333600
2,r1007,128000
3,r1008,190000
4,r1009,133800


In [16]:
sum_table.columns=['reserve_id','total_price_sum']

In [17]:
# 계산된 합을 결합하여 원본 테이블에 정보를 첨부한다.
# 합곗값이 없는 레코드의 합곗값을 fillna를 사용하여 0으로 변환
pd.merge(reserve_tb, sum_table, on='reserve_id',how='left').fillna(0).head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,total_price_sum
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,0.0
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,0.0
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,20600.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,0.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,0.0


--------------------
## 4.4 상호결합
고객의 월별 사용 금액을 집계하는 상황.
* step1. 고객 id별로 201701, 201702, 201703 날짜 컬럼을붙이고, 이후 Join을 위한 key값을 할당
* step2. reserve테이블에서 checkin_date는 년,월,일로 되어있으므로 이를 년,월 로 변경하여 새로운 컬럼 추가
* step3. step1에서 만들어놓은 테이블과 reserve테이블을 고객id와 년월값 을 key값으로하여 merge

> #### python skill
* relativedelta( )
    * datetime.date(2017,1,1)+ relativedelta(months=x) 로 월 단위로 날짜수를 더한것.
        * strftime("%Y%m") 함수로 연월의 문자열로 변환
* pd.to_datetime( )
    * 2016-03-26 형태를 201603로 변경

In [22]:
# 날짜형을 위한 라이브러리
import datetime
# 날짜를 계산하기 위한 라이브러리
from dateutil.relativedelta import relativedelta

In [63]:
# 연월 마스터 테이블 생성
month_mst = pd.DataFrame({
  'year_month':
    # relativedelta로 2017-01-01을 x월간 보낸다. x에는 0,1,2를 대입
    # 2017-01-01, 2017-02-01, 2017-03-01의 리스트 생성
    [(datetime.date(2017, 1, 1) + relativedelta(months=x)).strftime("%Y%m")        # strftime()함수로 연월의 문자열로 변환 (2017-01-01 을 201701로 변환)
     for x in range(0, 3)]
})
month_mst

,year_month
0,201701
1,201702
2,201703


In [32]:
# cross join을 위해 모두 같은 값을 가지는 결합 키를 준비
customer_tb['join_key']=0
month_mst['join_key']=0

In [35]:
customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude,join_key
0,c_1,41,man,35.092193,136.512347,0
1,c_2,38,man,35.325076,139.410551,0
2,c_3,49,woman,35.120543,136.511179,0
3,c_4,43,man,43.034868,141.240314,0
4,c_5,31,man,35.102661,136.523797,0


In [36]:
month_mst.head()

,year_month,join_key
0,201701,0
1,201702,0
2,201703,0


In [38]:
# customer_tb와 month_mst를 준비한 결합 키로 내부 결합하여 상호 결합을 구현한다.
### 고객 id별로 2017년 1,2,3월 날짜를 붙이기 위해 0값의 임시 키를 생성해서 merge한것.
customer_mst=pd.merge(customer_tb[['customer_id','join_key']], month_mst, on='join_key')
customer_mst.head()

,customer_id,join_key,year_month
0,c_1,0,201701
1,c_1,0,201702
2,c_1,0,201703
3,c_2,0,201701
4,c_2,0,201702


In [53]:
# 예약 테이블에서 연월의 결합 키를 준비
### checkin_date 컬럼에서 '년,월'만 뽑기
reserve_tb['year_month']=reserve_tb['checkin_date'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d').strftime("%Y%m"))
reserve_tb.head(10)

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,year_month
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,201603
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,201607
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,201610
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,201703
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,201709
5,r6,h_241,c_1,2017-11-27 18:47:05,2017-12-04,12:00:00,2017-12-06,3,36000,201712
6,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,201801
7,r8,h_241,c_1,2018-05-26 08:42:51,2018-06-08,10:00:00,2018-06-09,1,6000,201806
8,r9,h_217,c_2,2016-03-05 13:31:06,2016-03-25,09:30:00,2016-03-27,3,68400,201603
9,r10,h_240,c_2,2016-06-25 09:12:22,2016-07-14,11:00:00,2016-07-17,4,320400,201607


In [57]:
# 예약 레코드와 결합하여 사용 금액 합계를 계산.
summary_result=pd.merge(customer_mst, reserve_tb[['customer_id','year_month','total_price']],
                       on=['customer_id','year_month'], how='left').groupby(['customer_id','year_month'])['total_price'].sum().reset_index()
summary_result.head(10)

,customer_id,year_month,total_price
0,c_1,201701,0.0
1,c_1,201702,0.0
2,c_1,201703,194400.0
3,c_10,201701,0.0
4,c_10,201702,0.0
5,c_10,201703,0.0
6,c_100,201701,0.0
7,c_100,201702,18000.0
8,c_100,201703,0.0
9,c_1000,201701,0.0


In [58]:
summary_result.fillna(0, inplace=True)